In [9]:
f = open('dataset/regnie_dwd/ra031031', 'r')

In [10]:
lines=[]
for line in f:
    lines.append(repr(line))

In [33]:
len(lines)

972

In [29]:
len(lines[1][2:-3].split("-"))

611

In [70]:
file =[]

In [96]:
line = lines[50].replace("-"," ").replace("  "," ")[2:-3].split(" ")

In [101]:
line = [int(l) for l in  lines[50].replace("-"," ").replace("  "," ")[2:-3].split(" ") if l!=""]

In [102]:
len(line)

611

In [106]:
# one line of the grid that is 611x971 (west/east - northsouth)
file = []
for line in lines[:-1]:
    #print(line)
    file.append([int(l) for l in  line.replace("-"," ").replace("  "," ")[2:-3].split(" ") if l!=""])
#row1 = [int(i) for i in lines[1][2:-3].split("-")] 

In [109]:
file = np.array(file)

In [34]:
lines[-1]

"'    031031      3353 rus4 D            20041230\\n'"

In [55]:
xdelta = 1.0 * (1./60.)
ydelta = 1.0 * (1./120.)
mn =[(0,0),(971,0),(0,971),(971,971)]

In [56]:
# x:long , y:lat
for pt in mn:

    xmn = ( 6. - 10.*xdelta) + (pt[0] - 1)*xdelta
    ymn = (55. + 10.*ydelta) - (pt[1] - 1)*ydelta
    print(xmn,ymn)

5.816666666666666 55.09166666666667
22.0 55.09166666666667
5.816666666666666 47.0
22.0 47.0


### long/x: 5.8 west - 22 east
### lat/y: 55.09 north - 47 south

In [50]:
print(xmn,ymn)

5.816666666666666 55.09166666666667


In [65]:
x =[i for i in range(0,611)]
long =[( 6. - 10.*xdelta) + (pt - 1)*xdelta for pt in x]

In [66]:
y =[i for i in range(0,971)]
lat =[(55. + 10.*ydelta) - (pt - 1)*ydelta for pt in y]

In [68]:
from netCDF4 import Dataset
import numpy as np

root_grp = Dataset('py_netcdf4.nc', 'w', format='NETCDF4')
root_grp.description = 'Example simulation data'


In [69]:
# dimensions
root_grp.createDimension('time', None)
root_grp.createDimension('x', 611)
root_grp.createDimension('y', 971)


<class 'netCDF4._netCDF4.Dimension'>: name = 'y', size = 971

In [113]:
# variables
time = root_grp.createVariable('time', 'f8', ('time',))
x = root_grp.createVariable('x', 'f4', ('x',))
y = root_grp.createVariable('y', 'f4', ('y',))
field = root_grp.createVariable('field', 'f8', ('time', 'x', 'y',))

#data
x[:] = long
y[:] = lat
for i in range(5):
    time[i] = i*50.0
    field[i,:,:] = file


root_grp.close()

In [114]:
import xarray as xr
ds = xr.open_dataset('py_netcdf4.nc')
# index by value
#ds['temp'].sel(lon=65.5, lat=29.5)
# or index by position
#ds['temp'].isel(lon=5, lat=2)

In [155]:
prec = float(ds.field.sel(x=13.643653, y=  53.416793, time=100, method='nearest').values)
prec

48.0

In [156]:
type(prec)

float

#### extract all the dates in t s and put them in a list, then build nc file for all these days as layers, the we  slice over all days(time) at the same loc to create a t s, and store it in dict with its bgr_id

## apply the func on a df for all bgr stations